In [ ]:
!pip install symspellpy
!pip install emoji
!pip install gensim

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import spacy
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from symspellpy import SymSpell, Verbosity
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from difflib import get_close_matches
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from difflib import get_close_matches
from nltk.stem import WordNetLemmatizer
import string
from textblob import TextBlob
import emoji
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'symspellpy'

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv', on_bad_lines='skip')

In [ ]:
df = df[:5000]
df[['review', 'sentiment']].describe()

,review,sentiment
count,5000,5000
unique,4997,2
top,Quite what the producers of this appalling ada...,negative
freq,2,2532


## 1. Обработки

In [ ]:
# @title Основная функция препроцессинга текста без лемматизации

chat_words = {
"AFAIK": "As Far As I Know",
"AFK": "Away From Keyboard",
"ASAP": "As Soon As Possible",
"ATK": "At The Keyboard",
"ATM": "At The Moment",
"A3": "Anytime, Anywhere, Anyplace",
"BAK": "Back At Keyboard",
"BBL": "Be Back Later",
"BBS": "Be Back Soon",
"BFN": "Bye For Now",
"B4N": "Bye For Now",
"BRB": "Be Right Back",
"BRT": "Be Right There",
"BTW": "By The Way",
"B4": "Before",
"B4N": "Bye For Now",
"CU": "See You",
"CUL8R": "See You Later",
"CYA": "See You",
"FAQ": "Frequently Asked Questions",
"FC": "Fingers Crossed",
"FWIW": "For What It's Worth",
"FYI": "For Your Information",
"GAL": "Get A Life",
"GG": "Good Game",
"GN": "Good Night",
"GMTA": "Great Minds Think Alike",
"GR8": "Great!",
"G9": "Genius",
"IC": "I See",
"ICQ": "I Seek you (also a chat program)",
"ILU": "ILU: I Love You",
"IMHO": "In My Honest/Humble Opinion",
"IMO": "In My Opinion",
"IOW": "In Other Words",
"IRL": "In Real Life",
"KISS": "Keep It Simple, Stupid",
"LDR": "Long Distance Relationship",
"LMAO": "Laugh My A.. Off",
"LOL": "Laughing Out Loud",
"LTNS": "Long Time No See",
"L8R": "Later",
"MTE": "My Thoughts Exactly",
"M8": "Mate",
"NRN": "No Reply Necessary",
"OIC": "Oh I See",
"PITA": "Pain In The A..",
"PRT": "Party",
"PRW": "Parents Are Watching",
"QPSA?": "Que Pasa?",
"ROFL": "Rolling On The Floor Laughing",
"ROFLOL": "Rolling On The Floor Laughing Out Loud",
"ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
"SK8": "Skate",
"STATS": "Your sex and age",
"ASL": "Age, Sex, Location",
"THX": "Thank You",
"TTFN": "Ta-Ta For Now!",
"TTYL": "Talk To You Later",
"U": "You",
"U2": "You Too",
"U4E": "Yours For Ever",
"WB": "Welcome Back",
"WTF": "What The F...",
"WTG": "Way To Go!",
"WUF": "Where Are You From?",
"W8": "Wait...",
"7K": "Sick:-D Laugher",
"TFW": "That feeling when",
"MFW": "My face when",
"MRW": "My reaction when",
"IFYP": "I feel your pain",
"TNTL": "Trying not to laugh",
"JK": "Just kidding",
"IDC": "I don't care",
"ILY": "I love you",
"IMU": "I miss you",
"ADIH": "Another day in hell",
"ZZZ": "Sleeping, bored, tired",
"WYWH": "Wish you were here",
"TIME": "Tears in my eyes",
"BAE": "Before anyone else",
"FIMH": "Forever in my heart",
"BSAAW": "Big smile and a wink",
"BWL": "Bursting with laughter",
"BFF": "Best friends forever",
"CSL": "Can't stop laughing"
}

# Получаем список стоп-слов
nltk.download('stopwords')
stopword = stopwords.words('english')

def process_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление HTML-тегов
    text = re.sub(r'<.*?>', ' ', text)

    # Удаление URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Удаление пунктуации
    punc = string.punctuation
    text = text.translate(str.maketrans('', '', punc))

    # Замена чат-слов
    new_text = []
    for word in text.split():
        if word.upper() in chat_words:
            new_text.append(chat_words[word.upper()])
        else:
            new_text.append(word)
    text = " ".join(new_text)

    # Удаление стоп-слов и *лемматизация
    new_text = []
    for word in text.split():
        if word not in stopword:
            #word = lemmatizer.lemmatize(word)    #*После лемматизации результат незначительно, но ухудшался
            new_text.append(word)
    text = " ".join(new_text)

    # Замена эмодзи
    text = emoji.demojize(text)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# @title Функции создания словаря для исправлений

def get_word_frequencies(text, clean=True, remove_stopwords=True):
    if clean:
        text = re.sub(r'[^\w\s]', '', text.lower())
    words = text.split()
    if remove_stopwords:
        words = [word for word in words if word not in stopword]
    return Counter(words)

def correct_text_symspell(text):
    tokens = text.lower().split()
    corrected = []
    for token in tokens:
        suggestions = sym_spell.lookup(token, Verbosity.CLOSEST, max_edit_distance=2)
        corrected.append(suggestions[0].term if suggestions else token)
    return ' '.join(corrected)

In [ ]:
# @title Обработки текста с помощью NLTK

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def normalize_token(token):     #Нормализует токен, приводя его к нормальной форме и находя близкие совпадения.
    token_norm = token.lower()
    candidates = get_close_matches(token_norm, top_words, n=1, cutoff=0.85)
    return candidates[0] if candidates else token_norm

def normalize_text(text):     #Нормализует строку текста, разбивая ее на токены и нормализуя каждый токен.
    tokens = word_tokenize(text.lower())
    normalized = [normalize_token(t) for t in tokens if is_valid_token(t)]
    return ' '.join(normalized)

def nouns_only(text) -> str:    #Извлекает только существительные из текста
    tokens = word_tokenize(text)
    tagged = nltk.pos_tag(tokens, lang='eng')
    nouns = [token.lower() for token, pos in tagged if pos.startswith('N')]
    return ' '.join(nouns)

def remove_stopwords(text) -> str:      #Токенизирует текст, удаляет стоп-слова и оставляет только валидные токены.
    tokens = word_tokenize(text.lower())
    return ' '.join([t for t in tokens if t not in stopword and is_valid_token(t)])

def normalize_text(text) -> str:       #Удаляет цифры, пунктуацию и приводит текст к нижнему регистру.
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def preprocess_text(text):      #Лемматизирует текст, оставляя только существительные и прилагательные.
    tokens = word_tokenize(text)
    lemmas = []

    for token in tokens:
        lemma = lemmatizer.lemmatize(token.lower())
        lemmas.append(lemma)

    tagged = nltk.pos_tag(lemmas)
    filtered_lemmas = [lemma for lemma, pos in tagged if pos in ('NN', 'JJ')]

    return ' '.join(filtered_lemmas)

def is_valid_token(token):      #Проверяет, что токен не слишком короткий и не состоит только из цифр/знаков.
    if len(token) < 2:
        return False
    if re.match(r'^[\d\-\\/()]+', token):
        return False
    return True

def better_preprocess(text):        #Удаляет стоп-слова и бесполезные токены, но не лемматизирует.
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if is_valid_token(t)]
    return ' '.join(tokens) if tokens else text.lower()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# @title Обработки текста с помощью Spacy

nlp = spacy.load("en_core_web_sm")

def normalize_token(token):
    token_norm = token.lemma_
    candidates = get_close_matches(token_norm, top_words, n=1, cutoff=0.85)
    return candidates[0] if candidates else token_norm

def normalize_query(query):
    doc = nlp(query.lower())
    normalized = [normalize_token(token) for token in doc if is_valid_token(token.text)]
    return ' '.join(normalized)

def nouns_only(text) -> str:
    doc = nlp(text)
    nouns = [token.text.lower() for token in doc if token.pos_ == 'NOUN']
    return ' '.join(nouns)

def remove_stopwords(text) -> str:
    doc = nlp(text.lower())
    return ' '.join([token.text for token in doc if token.text not in stopword and is_valid_token(token.text)])

def normalize_text(text) -> str:
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def preprocess_text(text):
    doc = nlp(text)
    lemmas = []

    for token in doc:
        if token.pos_ in ('NOUN', 'ADJ'):
            lemmas.append(token.lemma_.lower())

    return ' '.join(lemmas)

def is_valid_token(token):
    if len(token) < 2:
        return False
    if re.match(r'^[\d\-\\/()]+', token):
        return False
    return True

def better_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if is_valid_token(token.text)]
    return ' '.join(tokens) if tokens else text.lower()

### 1.0 Обработка текста

In [ ]:
# Очищаем текст основным препроцессом

df['Processed_review'] = df['review'].apply(process_text)

In [ ]:
# Составляем словарь на очищенных данных.

combined_text = '\n'.join([
    ' '.join(df['Processed_review']).lower(),
])

combined_word_counts = get_word_frequencies(combined_text, clean=False, remove_stopwords=False) #Делаем уже на очищенных данных

word_freq_df = pd.DataFrame(combined_word_counts.items(), columns=['Word', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)

word_freq_df = word_freq_df[word_freq_df['Count'] > 1] #Обычно опечатки единичны, не заносим их в словарь

sym_spell = SymSpell(max_dictionary_edit_distance=2)

top_words = word_freq_df['Word'].tolist()

for index, row in word_freq_df.iterrows():
    sym_spell.create_dictionary_entry(row['Word'], row['Count'])

In [ ]:
# Исправляем ошибки по словарю

df['Corrected_review'] = df['Processed_review'].apply(correct_text_symspell)

In [ ]:
# Генерируем варианты обработок

df.loc[:, 'Lemma_noun-adj_review'] = df.loc[:, 'review'].apply(preprocess_text)
df.loc[:, 'Lemma_noun-adj_review'] = df.loc[:, 'Lemma_noun-adj_review'].replace('', pd.NA)
df.loc[:, 'Lemma_noun-adj_review'] = df.loc[:, 'Lemma_noun-adj_review'].fillna(df['review'])
df.loc[:, 'NoLemma_cleaned_review'] = df.loc[:, 'review'].apply(better_preprocess)
df.loc[:, 'NoLemma_noun_review'] = df.loc[:, 'review'].apply(nouns_only)
df.loc[:, 'NoLemma_light_cleaned_review'] = df.loc[:, 'review'].apply(remove_stopwords)
df.loc[:, 'NoLemma_dict_corrected_review'] = df.loc[:, 'review'].apply(normalize_text)

In [ ]:
# Если надо, всё в общую таблицу

melted_df = df.melt(id_vars=['sentiment'],
                             value_vars=[
                                         'review',
                                         'Processed_review',
                                         'Corrected_review',
                                         'Lemma_noun-adj_review',
                                         'NoLemma_cleaned_review',
                                         'NoLemma_noun_review',
                                         'NoLemma_light_cleaned_review',
                                         'NoLemma_dict_corrected_review'
                                         ],
                             var_name='ProcessingType',
                             value_name='ProcessingValue')

### 1.1 Пример обработки предложения

In [ ]:
sentence1 = "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."

In [ ]:
sentence1

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
sentence2 = process_text(sentence1)
sentence2

'one reviewers mentioned watching 1 oz episode youll hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far away would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle class inm

In [ ]:
correct_text_symspell(sentence2)

'one reviewers mentioned watching 1 oz episode youll hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangsta latinos christians italians irish moreso scuffle death stares dodgy dealings shady agreements never far away would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romance doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle class inmates

In [ ]:
preprocess_text(sentence1)

'other reviewer episode right first thing brutality scene violence right word show faint hearted timid show punch regard drug sex violence hardcore classic use word.<br />it oz nickname experimental section prison cell glass front inward privacy high agenda many gangsta irish more scuffle death stare dodgy dealing shady agreement away.<br main appeal show due fact other show pretty picture mainstream audience charm romance oz first episode nasty surreal ready more taste high level graphic violence violence injustice crooked guard nickel order mannered middle class prison bitch due lack street skill prison experience comfortable uncomfortable that touch dark side'

In [ ]:
better_preprocess(sentence1)

"one of the other reviewers has mentioned that after watching just oz episode you 'll be hooked they are right as this is exactly what happened with me.<br first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word.<br is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many .. aryans muslims gangstas latinos christians italians irish and more .... so scuffles death stares dodgy dealings and shady agreements are never far away.<br would say the main appeal of the show is due to the fact that it goes where other shows would n't dare forg

In [ ]:
nouns_only(sentence1)

'reviewers episode thing brutality scenes violence right word show show punches regards drugs sex violence hardcore use word.<br />it oz nickname section prison cells glass fronts inwards privacy agenda gangstas scuffles death stares dodgy dealings agreements away.<br appeal show fact shows pictures audiences charm romance oz episode taste levels violence violence injustice guards nickel order class prison bitches lack street skills prison experience thats touch side'

In [ ]:
remove_stopwords(sentence1)

"one reviewers mentioned watching oz episode 'll hooked right exactly happened me.<br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word.<br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home many .. aryans muslims gangstas latinos christians italians irish .... scuffles death stares dodgy dealings shady agreements never far away.<br would say main appeal show due fact goes shows would n't dare forget pretty pictures painted mainstream audiences forget charm forget romance ... oz n't mess around first episode ever saw struck nasty surreal could n't say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards 'll sold nickel inmates 'll kill order get away well mannered m

In [ ]:
normalize_text(sentence1)

'one of the other reviewers has mentioned that after watching just oz episode you ll be hooked they are right as this is exactly what happened with me br br the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word br br it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away br br i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t da

## 2. Векторизаторы и пайплайны

### 2.0 Подбор пайплайна

In [ ]:
# @title Tfidf Vectorizer

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

In [ ]:
# @title Count Vectorizer

from sklearn.feature_extraction.text import CountVectorizer

pipeline = Pipeline([
    ('vect', CountVectorizer()),  # заменили tfidf на count
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

In [ ]:
# @title Hashing Vectorizer

from sklearn.feature_extraction.text import HashingVectorizer

pipeline = Pipeline([
    ('vect', HashingVectorizer(alternate_sign=False)),  # alternate_sign=False важно для MultinomialNB
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

In [ ]:
# @title Gensim Vectorizer

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class GensimDoc2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, epochs=20):
        self.vector_size = vector_size
        self.epochs = epochs

    def fit(self, X, y=None):
        tagged_data = [TaggedDocument(words=doc.split(), tags=[i]) for i, doc in enumerate(X)]
        self.model = Doc2Vec(vector_size=self.vector_size, min_count=2, epochs=self.epochs)
        self.model.build_vocab(tagged_data)
        self.model.train(tagged_data, total_examples=self.model.corpus_count, epochs=self.model.epochs)
        return self

    def transform(self, X):
        return np.array([self.model.infer_vector(doc.split()) for doc in X])

pipeline = Pipeline([
    ('vect', GensimDoc2VecVectorizer()),
    ('clf', VotingClassifier(estimators=[
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

In [ ]:
# @title Spacy Vectorizer

from sklearn.base import BaseEstimator, TransformerMixin
import spacy
!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_md')

class SpacyVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model='en_core_web_md'):
        self.model = spacy.load(model)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([self.model(doc).vector for doc in X])

pipeline = Pipeline([
    ('vect', SpacyVectorizer()),
    ('clf', VotingClassifier(estimators=[
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 28.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# @title Transformer Vectorizer

from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class SentenceTransformerVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X, show_progress_bar=False)

pipeline = Pipeline([
    ('vect', SentenceTransformerVectorizer()),
    ('clf', VotingClassifier(estimators=[
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Error while downloading from https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/vocab.txt: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.
Trying to resume download...
Trying to resume download...


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

После серии тестов, лучшие результаты давала комбинация данных, построенная на чистых отзывах и очищенных + исправлениях по внутреннему словарю. В комментах оставил остальные варианты

In [ ]:
# @title Тесты векторизаторов

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold

total_f1 = []

kf = KFold(n_splits=5, shuffle=True, random_state=0)

X = df
y = df['sentiment']

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    X_train['sentiment'] = y_train.values

    melted_df = X_train.melt(id_vars=['sentiment'],
                             value_vars=[
                                         'review',
                                         #'Processed_review',
                                         'Corrected_review',
                                         #'Lemma_noun-adj_review',
                                         #'NoLemma_cleaned_review',
                                         #'NoLemma_noun_review',
                                         #'NoLemma_light_cleaned_review',
                                         #'NoLemma_dict_corrected_review'
                                         ],
                             var_name='ProcessingType',
                             value_name='ProcessingValue')

    pipeline.fit(melted_df['ProcessingValue'], melted_df['sentiment'])

    y_pred = pipeline.predict(X_test['Corrected_review'])

    f1_macro = f1_score(X_test['sentiment'], y_pred, average='macro')
    print("F1 Macro на тестовой выборке:", f1_macro)
    total_f1.append(f1_macro)
print("Ср.F1:", np.mean(total_f1))

**Сравнение векторизаторов** (на 5 фолдах, 5000 сэмплов):

| Векторизатор | F1-Score         | Скорость               |
|--------------|-----------------|-----------------------|
| TF-IDF       | 0.8701          | Очень быстро (<30 сек) |
| Count        | 0.8539          | Очень быстро (<30 сек)       |
| Hashing      | 0.8618          | Очень быстро (<30 сек) |
| Gensim       | 0.7930          | Медленно (10 мин)     |
| Spacy        | 0.7534          | Очень медленно (>30 мин) |
| Transformer  | 0.8042          | Очень медленно (>1 часа) |


### 2.1 TF-IDF + soft_voting(MNB + LogReg + LGB) + class_probas

Для эксперимента реализуем схему по выбору наиболее вероятного класса по предикту на 2 вариантах обработки (без обработки/с обработкой). Используем TF-IDF векторизацию как самую удачную

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('lr', LogisticRegression(class_weight='balanced')),
        ('lgb', lgb.LGBMClassifier(
        class_weight='balanced',
        objective='binary',
        learning_rate=0.05,
        num_leaves=31,
        n_estimators=100,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        verbose=-1
    )),
    ], voting='soft'))
])

In [ ]:
from sklearn.model_selection import KFold

before_f1 = []
total_f1 = []
results_df_list = []
proba_df_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=0)

X = df[['review', 'Corrected_review', 'sentiment']]
y = df['sentiment']

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train['sentiment'] = y_train.values

    melted_df = X_train.melt(id_vars=['sentiment'],
                             value_vars=[
                                         'review',
                                         'Corrected_review',
                                         ],
                             var_name='ProcessingType',
                             value_name='ProcessingValue')

    pipeline.fit(melted_df['ProcessingValue'], melted_df['sentiment'])

    # Посчитаем вероятность и категорию запроса по колонке 'review'
    X_test['review_sentiment'] = pipeline.predict(X_test['review'])
    y_proba = pipeline.predict_proba(X_test['review'])
    X_test['review_PredictedProbability'] = y_proba.max(axis=1)

    # Посчитаем вероятность и категорию запроса по колонке 'Corrected_review'
    X_test['Corrected_review_sentiment'] = pipeline.predict(X_test['Corrected_review'])
    y_proba = pipeline.predict_proba(X_test['Corrected_review'])
    X_test['Corrected_review_PredictedProbability'] = y_proba.max(axis=1)

    # Найдем индекс максимальной вероятности между всеми типами запросов
    max_proba_idx = np.argmax(X_test[[
                #'Processed_review_PredictedProbability',
                'review_PredictedProbability',
                'Corrected_review_PredictedProbability',
                ]].values, axis=1
    )

    # Соберем все предсказания в массив
    predicted_categories = X_test[[
        'review_sentiment',
        'Corrected_review_sentiment',
    ]].values

    # Соберем все вероятности в массив
    predicted_probas = X_test[[
        'review_PredictedProbability',
        'Corrected_review_PredictedProbability',
    ]].values

    # Получим финальное предсказание по максимальной вероятности и также добавим сами вероятности
    X_test['final_sentiment'] = [row[i] for row, i in zip(predicted_categories, max_proba_idx)]
    X_test['PredictedProbability'] = [row[i] for row, i in zip(predicted_probas, max_proba_idx)]

    f1_macro = f1_score(X_test['sentiment'], X_test['final_sentiment'], average='macro')
    print("F1 Macro на тестовой выборке:", f1_macro)
    total_f1.append(f1_macro)
print("Ср.F1:", np.mean(total_f1))

F1 Macro на тестовой выборке: 0.8697702747646849
F1 Macro на тестовой выборке: 0.8779409234069289
F1 Macro на тестовой выборке: 0.8638932923411955
F1 Macro на тестовой выборке: 0.8779687600025606
F1 Macro на тестовой выборке: 0.8639994559978239
Ср.F1: 0.8707145413026387


Результаты те же, улучшения или ухудшения нет при увеличении времени работы. Скип

## *3. All data test with final baseline

Далее будем экспериментировать с полным набором данных, будем делать только лёгкую обработку без лемматизации

In [ ]:
df_all = pd.read_csv('/content/IMDB Dataset.csv')

### 3.0 Предобработка

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)                         # HTML
    text = re.sub(r"\b\w{1,2}\b", " ", text)                   # Удалим короткие слова
    text = re.sub(r"[^a-z\s]", " ", text)                      # Только латинские буквы
    text = re.sub(r"\s+", " ", text).strip()                   # Лишние пробелы
    words = text.split()

    # Negation handling: заменим "not good" → "not_good"
    words = [f"{words[i]}_{words[i+1]}" if words[i] in ["not", "no", "never"] else words[i]
             for i in range(len(words) - 1)]

    # Стемминг и удаление стоп-слов
    words = [stemmer.stem(word) for word in words if word not in stop_words]

    return " ".join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Очищаем текст

df_all['Processed_review'] = df_all['review'].apply(clean_text)

### 3.1 Базовый TF-IDF + classifiers

In [ ]:
from sklearn.model_selection import KFold

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
    ngram_range=(1,3),
    max_features=50_000,
    sublinear_tf=True,
    norm='l2',
)),
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

total_f1 = []

kf = KFold(n_splits=5, shuffle=True, random_state=0)

X = df_all
y = df_all['sentiment']

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    X_train['sentiment'] = y_train.values

    melted_df = X_train.melt(id_vars=['sentiment'],
                             value_vars=[
                                         #'review',
                                         'Processed_review'
                                         ],
                             var_name='ProcessingType',
                             value_name='ProcessingValue')

    pipeline.fit(melted_df['ProcessingValue'], melted_df['sentiment'])

    y_pred = pipeline.predict(X_test['Processed_review'])

    f1_macro = f1_score(X_test['sentiment'], y_pred, average='macro')
    print("F1 Macro на тестовой выборке:", f1_macro)
    total_f1.append(f1_macro)
print("Ср.F1:", np.mean(total_f1))

Ранее был проведён тест на чистых review (f1=0.9121 / 8 мин) и на комбинации (f1=0.9001 / 13 мин). Лучший вариант в соотношении скорость/точность - только на обработанном тексте (f1=0.9055 / 5 мин)

### 3.2 TF-IDF + feature selection

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Предположим: reviews, labels — твои данные
X_train, X_test, y_train, y_test = train_test_split(df_all['Processed_review'], df_all['sentiment'], test_size=0.2)

# Векторизация
tfidf = TfidfVectorizer(
    preprocessor=clean_text,
    ngram_range=(1, 3),
    max_features=50000,
    sublinear_tf=True,
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Feature selection
selector = SelectKBest(mutual_info_classif, k=20000)
X_train_sel = selector.fit_transform(X_train_tfidf, y_train)
X_test_sel = selector.transform(X_test_tfidf)

# Классификатор
clf = LogisticRegression(max_iter=200)
clf.fit(X_train_sel, y_train)
y_pred = clf.predict(X_test_sel)

print("TF-IDF + Mutual Info Accuracy:", accuracy_score(y_test, y_pred))

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values f

TF-IDF + Mutual Info Accuracy: 0.8859


In [ ]:
clf = Pipeline([
    ('clf', VotingClassifier(estimators=[
        ('mnb', MultinomialNB(alpha=0.5)),
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

clf.fit(X_train_sel, y_train)
y_pred = clf.predict(X_test_sel)

print("TF-IDF + Mutual Info Accuracy:", accuracy_score(y_test, y_pred))

TF-IDF + Mutual Info Accuracy: 0.8903


Неплохо, но хуже простого варианта + долго (44 минуты на 1 фолд)

### 3.3 FastText

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gunzip cc.en.300.vec.gz

--2025-06-19 14:02:20--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.115, 18.238.176.44, 18.238.176.126, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  71.9MB/s    in 22s     

2025-06-19 14:02:43 (57.3 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [ ]:
from gensim.models import KeyedVectors
import numpy as np

# Загрузка (один раз, может занять 2–3 мин)
ft_model = KeyedVectors.load_word2vec_format("/content/cc.en.300.vec", binary=False)

def fasttext_vector(text):
    words = clean_text(text).split()
    vectors = [ft_model[word] for word in words if word in ft_model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)

X_train_vec = np.array([fasttext_vector(text) for text in X_train])
X_test_vec = np.array([fasttext_vector(text) for text in X_test])

clf = LogisticRegression(max_iter=200)
clf.fit(X_train_vec, y_train)
print("FastText Accuracy:", accuracy_score(y_test, clf.predict(X_test_vec)))

FastText Accuracy: 0.7972


In [ ]:
clf = Pipeline([
    ('clf', VotingClassifier(estimators=[
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

clf.fit(X_train_vec, y_train)
print("FastText Accuracy:", accuracy_score(y_test, clf.predict(X_test_vec)))

FastText Accuracy: 0.8049


Скорее всего, тут не подходит даже с настройкой параметров

### 3.4 sBERT

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # paraphrase-MiniLM-L12-v2

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

X_train_sbert = model.encode(X_train, batch_size=32, show_progress_bar=True)
X_test_sbert = model.encode(X_test, batch_size=32, show_progress_bar=True)

clf = LogisticRegression(max_iter=300)
clf.fit(X_train_sbert, y_train)
y_pred = clf.predict(X_test_sbert)

print("sBERT + LR Accuracy:", accuracy_score(y_test, y_pred))

Batches:   0%|          | 0/1250 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

sBERT + LR Accuracy: 0.7754


In [ ]:
clf = Pipeline([
    ('clf', VotingClassifier(estimators=[
        ('rcl', RidgeClassifier(alpha=0.5, tol=0.1)),
        ('pac', PassiveAggressiveClassifier(C=0.1, tol=0.5)),
        ('sgd', SGDClassifier(alpha=0.0001, epsilon=0.1)),
    ], voting='hard'))
])

clf.fit(X_train_sbert, y_train)
y_pred = clf.predict(X_test_sbert)

print("sBERT + LR Accuracy:", accuracy_score(y_test, y_pred))

sBERT + LR Accuracy: 0.7781


Аналогично плохой результат. Специфика не для классификации текстов, а для подбора похожих текстов либо генерации новых на основе имеющихся